## Bibliotecas 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
df = pd.read_csv("vehicles.csv", index_col="id")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Understanding my variables

In [8]:
df.shape
df.head()
df.columns

Index(['url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model',
       'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

## Better understanding of the different values for each variable.

In [11]:
#.nunique(axis=0) returns the number of unique values for each variable.
df.nunique(axis=0)

url             426880
region             404
region_url         413
price            15655
year               114
manufacturer        42
model            29667
condition            6
cylinders            8
fuel                 5
odometer        104870
title_status         6
transmission         3
VIN             118264
drive                3
size                 4
type                13
paint_color         12
image_url       241899
description     360911
county               0
state               51
lat              53181
long             53772
posting_date    381536
dtype: int64

In [3]:
# .describe() summarizes the count, mean, standard deviation, min, and max for numeric variables. The code that follows this simply formats 
#each row to the regular format and suppresses scientific notation

df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

,price,year,odometer,county,lat,long
count,426880.000000,425675.000000,422480.000000,0.000000,420331.000000,420331.000000
mean,75199.033187,2011.235191,98043.331443,nan,38.493940,-94.748599
std,12182282.173598,9.452120,213881.500798,nan,5.841533,18.365462
min,0.000000,1900.000000,0.000000,nan,-84.122245,-159.827728
25%,5900.000000,2008.000000,37704.000000,nan,34.601900,-111.939847
50%,13950.000000,2013.000000,85548.000000,nan,39.150100,-88.432600
75%,26485.750000,2017.000000,133542.500000,nan,42.398900,-80.832039
max,3736928711.000000,2022.000000,10000000.000000,nan,82.390818,173.885502


In [2]:
#contrib_df["AMNT"].describe().apply(lambda x: format(x, 'f'))

In [5]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,price,year,odometer,county,lat,long
count,426880.00000,425675.00000,422480.00000,0.00000,420331.00000,420331.00000
mean,75199.03319,2011.23519,98043.33144,nan,38.49394,-94.74860
std,12182282.17360,9.45212,213881.50080,nan,5.84153,18.36546
min,0.00000,1900.00000,0.00000,nan,-84.12225,-159.82773
25%,5900.00000,2008.00000,37704.00000,nan,34.60190,-111.93985
50%,13950.00000,2013.00000,85548.00000,nan,39.15010,-88.43260
75%,26485.75000,2017.00000,133542.50000,nan,42.39890,-80.83204
max,3736928711.00000,2022.00000,10000000.00000,nan,82.39082,173.88550


## Using .unique(), I took a look at my discrete variables, including ‘condition’.

In [3]:
df.condition.unique()

NameError: name 'df' is not defined

You can see that there are many synonyms of each other, like ‘excellent’ and ‘like new’. While this isn’t the greatest example, there will be some instances where it‘s ideal to clump together different words. For example, if you were analyzing weather patterns, you may want to reclassify ‘cloudy’, ‘grey’, ‘cloudy with a chance of rain’, and ‘mostly cloudy’ simply as ‘cloudy’.

Later you’ll see that I end up omitting this column due to having too many null values, but if you wanted to re-classify the condition values, you could use the code below:

In [16]:
# Reclassify condition column
def clean_condition(row):
    
    good = ['good','fair']
    excellent = ['excellent','like new']       
    
    if row.condition in good:
        return 'good'   
    if row.condition in excellent:
        return 'excellent'    
    return row.condition# Clean dataframe
def clean_df(playlist):
    df_cleaned = df.copy()
    df_cleaned['condition'] = df_cleaned.apply(lambda row: clean_condition(row), axis=1)
    return df_cleaned# Get df with reclassfied 'condition' column
df_cleaned = clean_df(df)

print(df_cleaned.condition.unique())

[nan 'good' 'excellent' 'new' 'salvage']


# 2. Cleaning your dataset

## a. Removing Redundant variables

In [19]:
# First I got rid of variables that I thought were redundant. This includes url, image_url, and city_url.
df_cleaned = df_cleaned.copy().drop(['url','image_url','region_url'], axis=1)

## b. Variable Selection

In [22]:
NA_val = df_cleaned.isna().sum()

def na_filter(na, threshold = .4): #only select variables that passees the threshold
    col_pass = []
    for i in na.keys():
        if na[i]/df_cleaned.shape[0]<threshold:
            col_pass.append(i)
    return col_pass
df_cleaned = df_cleaned[na_filter(NA_val)]
df_cleaned.columns

Index(['region', 'price', 'year', 'manufacturer', 'model', 'fuel', 'odometer',
       'title_status', 'transmission', 'VIN', 'drive', 'type', 'paint_color',
       'description', 'state', 'lat', 'long', 'posting_date'],
      dtype='object')

## c. Removing Outliers

In [23]:
df_cleaned = df_cleaned[df_cleaned['price'].between(999.99, 99999.00)]
df_cleaned = df_cleaned[df_cleaned['year'] > 1990]
df_cleaned = df_cleaned[df_cleaned['odometer'] < 899999.00]df_cleaned.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

SyntaxError: invalid syntax (1204710063.py, line 3)